# Creation of maps with the extracted datasets

In [106]:
import os
import fileinput

import numpy as np
import pandas as pd
import geopandas as gpd
import pickle
from tqdm import tqdm
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import shapely.wkt
from shapely.geometry import Point, Polygon
import pysal

import re

from scipy import stats

from branca.colormap import linear

import folium
import folium.plugins as plugins
from folium.plugins import MarkerCluster, FastMarkerCluster, HeatMapWithTime, TimeSliderChoropleth
from folium import IFrame

## Create basic event map with geopandas

In [ ]:
with open(os.path.join("pickle", 'conflict.pickle'), 'rb') as data_source:
    conflict_df = pickle.load(data_source)

In [ ]:
geometry = conflict_df['geom_wkt'].map(shapely.wkt.loads)
conflict_df = conflict_df.drop('geom_wkt', axis=1)
crs = {'init': 'epsg:4326'}
conflict_gdf = gpd.GeoDataFrame(conflict_df, crs=crs, geometry=geometry)


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


world.plot(ax=ax, color='white', edgecolor='black')
conflict_gdf.plot(ax=ax, markersize=(conflict_gdf.loc[:, "best"]/1000), cmap="autumn")
    
plt.show()

## Create heatmap

### Generate the event_list dataframe

In [ ]:
display(conflict_df.head(1))

In [ ]:
event_list = conflict_df[["id", "latitude", "longitude", "deaths_civilians", "best", "year"]]
display(event_list.head(2))

In [ ]:
# keep only events w/ deaths
event_list = event_list[event_list.best > 0]

In [ ]:
event_list = event_list.sort_values("year")

In [ ]:
event_list.sort_values("best", ascending = False)
display(event_list.sort_values("best").head(5))

### Create Heatmap with each events layered by year

In [ ]:
event_df_array = []
for year in np.sort(event_list.year.unique()):
    print(year, end=", ") #make sure years are in the desired order
    event_df_array.append(event_list[event_list.year == year])

In [ ]:
event_pos_date_array = []
for index in range(len(event_df_array)):
    event_pos_date_array.append([[row.latitude, row.longitude] for row in event_df_array[index].itertuples()])

In [ ]:
m = folium.Map(tiles='cartodbpositron', world_copy_jump = True, no_wrap=True, zoom_start=8)

index = 0
# TODO: find good gradient color for colorblind that still show the heatmap
for index, year in enumerate(event_list.year.unique()):
    m.add_child(plugins.HeatMap(event_pos_date_array[index],
                                name="{}".format(year),
                                max_val=event_list.best.max(),
                                min_opacity=0.99,
                                radius=2,
                                blur=3,
                                max_zoom=10,
                                gradient ={0.01: 'blue', 0.04: 'cyan', 0.07: 'green', 0.15: 'yellow', 0.3:'orange', 0.5:'red',  1: 'magenta'},
                                overlay=True))


folium.LayerControl().add_to(m)
folium.plugins.Fullscreen().add_to(m)
m.save(os.path.join("results", 'heatmap_w_time_layered.html'))

### Create a timestamped heatmap

In [ ]:
NBR_OF_BINS=1000
binned_deaths, bins = pd.qcut(event_list.best, q=NBR_OF_BINS, duplicates='drop', labels=False, retbins=True)
binned_deaths.name="bin_deaths"
# We add 1 to avoid bins at 0, because they would have null weight
binned_deaths += 1
binned_deaths_df = pd.DataFrame(binned_deaths )
print(len(bins))
display(binned_deaths_df.head(5))
# Avoid doing the merge multiple times
if "bin_deaths" not in event_list.columns:
    event_list = event_list.merge(binned_deaths_df,left_index=True, right_index=True )

In [ ]:
# NOT USEFULL SECTION FOR NOW! MIGHT BE USEFULL TO SHOW A LEGEND OR A BIN GRAPH
bin_df_cols = ["bin", "bin_range" , "nbr"]
heatmap_bins_df = pd.DataFrame([], columns=bin_df_cols).set_index(bin_df_cols[0])

for index, value in enumerate(bins):
    min_val = bins[index-1] if index is not 0 else 0
    max_val = bins[index]
    bin_range = (min_val, max_val)
    nbr =  len(event_list.loc[event_list.bin_deaths == (index+1)])
    temp_bin_df = pd.DataFrame([[index, bin_range, nbr]], columns=bin_df_cols)
    temp_bin_df = temp_bin_df.set_index(bin_df_cols[0])
    heatmap_bins_df = heatmap_bins_df.append(temp_bin_df)
display(heatmap_bins_df.head(5))

In [ ]:
m = folium.Map(tiles='cartodbpositron', world_copy_jump = True, no_wrap=True, zoom_start=8)

# We used bin dataset because it filters the extreme values, especially on the maximum size
# The norm isn't a 0 to 1 norm, it is a 1/bin_deaths.max() to 1, to force every event to be showed
column_to_use = "bin_deaths"
best_max = event_list[column_to_use].max()
best_min = event_list[column_to_use].min()
get_norm = lambda best: (best)/(best_max)



data_to_show = []
time_array = []
for year in event_list.year.unique():
    time_array.append(year)
    year_data = [[row.latitude, row.longitude, get_norm(row.bin_deaths)] for row in event_list.loc[event_list.year == year].itertuples()]
    data_to_show.append(year_data)
    
hm = plugins.HeatMapWithTime(data=data_to_show,
                             index=time_array,
                             name="deaths heatmap",
                             radius=5,
                             min_opacity=0.3,
                             max_opacity=0.9,
                             #scale_radius=True,
                             use_local_extrema=True,
                             display_index=True,
                             auto_play=True,
                             speed_step=0.1,
                             )
hm.add_to(m)

folium.plugins.Fullscreen().add_to(m)
m.save(os.path.join("results", 'heatmap_w_time.html'))

## Create maps with choroplets and events superposed using GeoPandas

### Create human cost GeoDataFrame

In [9]:
with open(os.path.join("pickle", 'country_human_cost.pickle'), 'rb') as data_source:
    country_human_cost_df = pickle.load(data_source)

In [11]:
display(country_human_cost_df.head(5))

number_of_events                         events_id  \
year   country_id                                                       
1989.0 2.0                      0.0                                 0   
       20.0                     0.0                                 0   
       31.0                     0.0                                 0   
       40.0                     0.0                                []   
       41.0                     4.0  [186087, 186122, 186123, 186124]   
       42.0                     0.0                                 0   
       51.0                     1.0                          [107752]   
       52.0                     0.0                                 0   
       70.0                     2.0                  [182047, 182048]   
       80.0                     0.0                                 0   

                   displacement_extern  displacement_intern  \
year   country_id                                             
1989.0 2.0                         0.0                  0.0   
       20.0                        0.0                  0.0   
       31.0                        0.0                  0.0   
       40.0                     7682.0                  0.0   
       41.0                     3344.0                  0.0   
       42.0                        0.0                  0.0   
       51.0                        0.0                  0.0   
       52.0                        0.0                  0.0   
       70.0                        0.0                  0.0   
       80.0                        0.0                  0.0   

                   total_displacement  deaths           gdp  hdi country_code  
year   country_id                                                              
1989.0 2.0                        0.0     0.0  22599.992143  0.0          USA  
       20.0                       0.0     0.0  20700.799311  0.0          CAN  
       31.0                       0.0     0.0  14213.543372  0.0          BHS  
       40.0                    7682.0     0.0   2577.207019  0.0          CUB  
       41.0                    3344.0    48.0    393.347528  0.0          HTI  
       42.0                       0.0     0.0   2604.425557  0.0          DOM  
       51.0                       0.0     1.0   1945.132761  0.0          JAM  
       52.0                       0.0     0.0   3562.104294  0.0          TTO  
       70.0                       0.0     3.0   2967.677913  0.0          MEX  
       80.0                       0.0     0.0   1979.342672  0.0          BLZ

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
world.head(5)

In [ ]:
indexed_world = world.set_index(["iso_a3"])
display(indexed_world.head(5))

In [ ]:
geometry = pd.DataFrame([], columns=["year", "country_id", "geometry"])
for row in country_human_cost_df.itertuples():
    geometry_val = indexed_world.loc[row.country_code, "geometry"]
    temp_geo = pd.DataFrame([[row.Index[0], row.Index[1], geometry_val]], columns=["year", "country_id", "geometry"])
    geometry = pd.concat([geometry, temp_geo])
display(geometry.head(5))

In [ ]:
geometry = geometry.set_index(["year", "country_id"])

In [ ]:
crs = {'init': 'epsg:4326'}
human_cost_gdf = gpd.GeoDataFrame(country_human_cost_df, crs=crs, geometry=geometry.geometry.values)

In [ ]:
display(human_cost_gdf.head(3))

In [ ]:
display(conflict_gdf.head(3))

In [ ]:
nbr_of_years = len(conflict_gdf.year.unique())
print(nbr_of_years)

In [ ]:
def gen_map_event_with_col(human_cost_gdf, conflict_gdf, column="total_displacement", name="map_event_deaths"):
    fig, ax = plt.subplots(ncols=2, nrows=14, figsize=(20,100))

    for centered_year in tqdm(range(nbr_of_years)):
        row = centered_year // 2
        col = centered_year % 2
        conflict_gdf_subset = conflict_gdf.loc[conflict_gdf.year == 1989+centered_year]
        world.plot(ax=ax[row, col], color='white', edgecolor='black')
        human_cost_gdf.loc[1989+centered_year].plot(ax=ax[row, col], column=column, cmap="GnBu", scheme='quantiles', legend=column)
        conflict_gdf_subset.plot(ax=ax[row, col], markersize=np.log2(conflict_gdf_subset.loc[:, "best"]), alpha=0.5, cmap="YlOrRd", legend="Event Deaths")
        ax[row, col].legend(pos=3) 
        ax[row, col].set_title("{}".format(1989+centered_year))
    fig.savefig(os.path.join("results", "{}_{}.png".format(name,column)))
    fig.clf()


In [ ]:
# gen maps for total displacement and gdp with events
gen_map_event_with_col(human_cost_gdf, conflict_gdf, column="total_displacement", name="map_event_deaths")
gen_map_event_with_col(human_cost_gdf, conflict_gdf, column="gdp", name="map_event_deaths")

In [ ]:
# Calculate data correlations
print("Total_displacement")
print("Pearson: ", stats.stats.pearsonr(human_cost_gdf["total_displacement"], human_cost_gdf["deaths"]))
print("Spearman: ", stats.stats.spearmanr(human_cost_gdf["total_displacement"], human_cost_gdf["deaths"]))
print("GDP")
print("Pearson: ", stats.stats.pearsonr(human_cost_gdf["gdp"], human_cost_gdf["deaths"]))
print("Spearman: ", stats.stats.spearmanr(human_cost_gdf["gdp"], human_cost_gdf["deaths"]))

### Plot by conflict instead of singular event

In [ ]:
new_conflict_gdf = conflict_gdf[["year", "conflict_new_id", "best", "geometry" ]]
display(new_conflict_gdf.head(5))

In [ ]:
dissolved_conflict_gdf = new_conflict_gdf.dissolve(by=["year", "conflict_new_id"], aggfunc='sum').reset_index()
display(dissolved_conflict_gdf.head(5))

In [ ]:
new_geometry = dissolved_conflict_gdf["geometry"].apply(lambda x: x if type(x) is Point or len(x) < 3 else Polygon([(p.x, p.y) for p in x]))
dissolved_conflict_gdf.loc[:, "geometry"] = new_geometry
display(dissolved_conflict_gdf.head(5))

In [ ]:
# gen maps for total displacement and gdp with conflicts
gen_map_event_with_col(human_cost_gdf, dissolved_conflict_gdf, column="total_displacement", name="map_conflict_deaths")
gen_map_event_with_col(human_cost_gdf, dissolved_conflict_gdf, column="gdp", name="map_conflict_deaths")

## Create interactive event + metric maps with folium

### Use time slider

In [55]:
dt_index = pd.date_range(start=pd.datetime(1989, 1, 1), periods=28, freq='AS').strftime('%s')
year_to_dt_index_dict = {}
for year in range(len(dt_index)):
    year_to_dt_index_dict[year+1989] = dt_index[year]
display(dt_index)

array(['599612400', '631148400', '662684400', '694220400', '725842800',
       '757378800', '788914800', '820450800', '852073200', '883609200',
       '915145200', '946681200', '978303600', '1009839600', '1041375600',
       '1072911600', '1104534000', '1136070000', '1167606000',
       '1199142000', '1230764400', '1262300400', '1293836400',
       '1325372400', '1356994800', '1388530800', '1420066800', '1451602800'],
      dtype='<U10')

{1989: '599612400',
 1990: '631148400',
 1991: '662684400',
 1992: '694220400',
 1993: '725842800',
 1994: '757378800',
 1995: '788914800',
 1996: '820450800',
 1997: '852073200',
 1998: '883609200',
 1999: '915145200',
 2000: '946681200',
 2001: '978303600',
 2002: '1009839600',
 2003: '1041375600',
 2004: '1072911600',
 2005: '1104534000',
 2006: '1136070000',
 2007: '1167606000',
 2008: '1199142000',
 2009: '1230764400',
 2010: '1262300400',
 2011: '1293836400',
 2012: '1325372400',
 2013: '1356994800',
 2014: '1388530800',
 2015: '1420066800',
 2016: '1451602800'}

In [ ]:
# First we start with the chloropleth maps, its easier to manage
display(human_cost_gdf.head(1))

In [ ]:
def year_list_to_dt_index(year_list):
    indexes = np.subtract(year_list, 1989).astype(int)
    values = dt_index[indexes]
    return values

In [ ]:
# We will use constant opacity and variable color to represent the metrics
def get_colormap(df, column="gdp"):
    min_value = df[column].min() 
    max_value = df[column].max() 
    cmap = linear.GnBu.scale(min_value, max_value)
    return cmap

cmap_gdp = get_colormap(human_cost_gdf, "gdp")
cmap_disp = get_colormap(human_cost_gdf, "total_displacement")
cmap_deaths = get_colormap(human_cost_gdf, "deaths")

In [ ]:
display(human_cost_gdf.index.get_level_values(1))
human_cost_gdf.sort_index(inplace=True)

In [ ]:
def get_style_data(df, column="gdp", opacity=0.7, ):
    cmap = get_colormap(df, column)
    style_data_dict = {}
    nbr_of_years = len(dt_index)
    min_color_value = "#ffffffff" 
    for country in df.index.get_level_values(1).unique():
        
        years = df.loc[(slice(None), country) , :].index.remove_unused_levels().get_level_values(0)
        
        partial_color_list = df.loc[(slice(None), country), column].apply(cmap).values
        partial_opacity_list = np.full(nbr_of_years, opacity)
        color_list = np.full(nbr_of_years, min_color_value)
        for index, year in enumerate(years):
            color_list[np.where(dt_index == year_to_dt_index_dict[year])] = partial_color_list[index]
        opacity_list = np.full(nbr_of_years, opacity)
        
        local_df = pd.DataFrame(
            {'color' : color_list,
             'opacity' : opacity_list},
             index=dt_index
        )
        local_df = local_df.sort_index()
        style_data_dict[str(int(country))] = local_df.to_dict(orient='index')
    return style_data_dict

In [ ]:
only_country_gdf = human_cost_gdf.copy()
try:
    only_country_gdf.index = only_country_gdf.index.droplevel("year")
except: # If we rerun
    pass
only_country_gdf = only_country_gdf.iloc[~only_country_gdf.index.duplicated()].sort_index()
only_country_gdf = only_country_gdf.reset_index()
only_country_gdf["country_id"] = only_country_gdf["country_id"].astype(int)
only_country_gdf = only_country_gdf.set_index("country_id")
display(only_country_gdf.head(5))

In [ ]:
style_dict_gdp = get_style_data(human_cost_gdf, "gdp", 1.0)
print(len(style_dict_gdp))
display(style_dict_gdp)

In [ ]:
only_country_gdf = only_country_gdf[["total_displacement", "deaths", "gdp", "geometry"]]
only_country_json = only_country_gdf.to_json() 
print(len(only_country_gdf))

In [ ]:
m = folium.Map(tiles='cartodbpositron', world_copy_jump = True, no_wrap=True, zoom_start=5)
slider_gdp = plugins.TimeSliderChoropleth(
    only_country_json,
    styledict = get_style_data(human_cost_gdf, "gdp", 0.7),
    name="gdp"
).add_to(m)
folium.LayerControl().add_to(m)
folium.plugins.Fullscreen().add_to(m)
m.save(os.path.join("results", "gdp_time_slider.html"))
m

This result is interesting, but the time slider is somewhat buggy and we can't add the event data to the timeline... we need to find another solution

### Use TimeStamped GeoJson

In [23]:
# Time stamped GeoJson makes it possible to use custom js for each event and for each country
# TODO:
# 2) Recreate the timeslider with colors for country with TS.GJ
# 3) Add the events to the TS.GJ
# 4) Try to split them into different layers

In [379]:
with open(os.path.join("pickle", 'distance_nx.pickle'), 'rb') as data_source:
    distance_graph = pickle.load(data_source)

In [380]:
conflict_graph_df = pd.DataFrame([])
for node in tqdm(distance_graph.nodes()):
    if distance_graph.node[node]["nature"] == "event":
        continue
    for year in distance_graph.node[node]["yearly_dict"]:
        conflict_graph_df = conflict_graph_df.append(distance_graph.node[node]["yearly_dict"][year],ignore_index=True)
display(conflict_graph_df.head(10))

100%|██████████| 136178/136178 [00:13<00:00, 9791.53it/s]   


,best,conflict_id,deaths_a,deaths_b,deaths_civilians,gwnoa,gwnob,latitude,longitude,side_a,side_b,type_of_violence,year
0,175.0,230.0,109.0,51.0,3.0,678.0,NaN,14.257709,45.931985,Government of Yemen (North Yemen),AQAP,1.0,2010.0
1,1140.0,230.0,398.0,658.0,42.0,678.0,NaN,13.457949,45.633967,Government of Yemen (North Yemen),AQAP,1.0,2011.0
2,2330.0,230.0,672.0,1534.0,45.0,678.0,NaN,13.953383,45.794770,Government of Yemen (North Yemen),AQAP,1.0,2012.0
3,94.0,230.0,5.0,48.0,41.0,678.0,NaN,15.051622,46.343938,Government of Yemen (North Yemen),AQAP,1.0,2009.0
4,582.0,230.0,207.0,282.0,35.0,678.0,NaN,14.564638,46.561987,Government of Yemen (North Yemen),AQAP,1.0,2013.0
5,1660.0,230.0,578.0,905.0,54.0,678.0,NaN,14.727340,46.227271,Government of Yemen (North Yemen),AQAP,1.0,2014.0
6,6532.0,230.0,2706.0,940.0,1483.0,678.0,NaN,14.580992,44.481574,Government of Yemen (North Yemen),AQAP,1.0,2015.0
7,2536.0,230.0,810.0,528.0,585.0,678.0,NaN,15.544922,44.270784,Government of Yemen (North Yemen),Forces of Hadi,1.0,2016.0
8,142.0,715.0,0.0,0.0,89.0,678.0,NaN,13.941369,44.424773,Government of Yemen (North Yemen),Civilians,3.0,2011.0
9,1489.0,402.0,515.0,71.0,308.0,678.0,NaN,13.428009,45.247445,Government of Yemen (North Yemen),Democratic Republic of Yemen,1.0,1994.0


#### Get circle icons features for every events

In [502]:
def get_circle_icon_style_from_series(pd_series, opacity=0.7, color='green', radius_scale=1.0):
    """Get the icon_style to create circles with a radius influenced by the value
       TODO: add option to cmap the color field also
    """
    radius_array = list(pd_series.map(lambda val: radius_scale*np.log(val) if val != 0 else 0))
    icon_style_dict_array = []
    for index, val in tqdm(enumerate(radius_array)):
        icon_style_base_dict = {}
        icon_style_base_dict['fillColor'] = color
        icon_style_base_dict['fillOpacity'] = opacity
        icon_style_base_dict['opacity'] = 0
        local_dict = icon_style_base_dict
        local_dict['radius'] = np.asscalar(radius_array[index])
        icon_style_dict_array.append(local_dict)
    return icon_style_dict_array

In [503]:
def get_feature_circle_array(df, columns=["best", "deaths_civilians"], opacity =[0.5, 1], color=["red", "yellow"], radius_scale=[1.0, 1.0]):
    """Get the feature array for the circles. Each circles has its own params"""
    circle_style_array = []
    for index, column in tqdm(enumerate(columns)):
        circle_style_array.append(get_circle_icon_style_from_series(df[column], opacity=opacity[index], color=color[index], radius_scale=radius_scale[index]))
    # Create associated Features
    features = [[], []]
    for row in tqdm(df.itertuples()):
        for index in range(len(columns)):
            start = "{}-01-01".format(int(row.year))
            end = "{}-01-01".format(int(row.year)+1)
            local_marker_feature_dict = {
                                   "type": "Feature",
                                   "geometry": {"type": "Point",
                                                "coordinates": [row.longitude, row.latitude]
                                               },
                                   "properties" : {"time": "{}-01-01".format(int(row.year)),
                                                   "icon": "circle",
                                                   "iconstyle": circle_style_array[index][row.Index],
                                                   "id":"cir{}_{}".format(row.Index, index)
                                   }
            }
            features[index].append(local_marker_feature_dict)
    return features

In [526]:
feature_circle_array = get_feature_circle_array(
    conflict_graph_df,
    columns=["best", "deaths_civilians"],
    opacity =[0.8, 1],
    color=["red", "blue"],
    radius_scale=[1, 1]
    )

0it [00:00, ?it/s]
0it [00:00, ?it/s]
4865it [00:00, 670014.41it/s]
0it [00:00, ?it/s]
2it [00:00, 61.09it/s]92it/s]
4865it [00:00, 110923.63it/s]


In [527]:
print(feature_circle_array[1][1])

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [45.633966856321884, 13.45794864942529]}, 'properties': {'time': '2011-01-01', 'icon': 'circle', 'iconstyle': {'fillColor': 'blue', 'fillOpacity': 1, 'opacity': 0, 'radius': 3.7376696182833684}, 'id': 'cir1_1'}}


#### Get popup marker icons features for every events

In [528]:
def extract_gnwo_countries_to_df():
    """Extract the countries from the gnwo and their id from the gnwo files"""
    countries_list = []
    # First gnwo file, contains id, code, name for all gnow numbers
    with open(os.path.join("data", "gnwo.txt"), "r") as gnow:
        for line in gnow:
            split_line = re.split(r'\t+', line)
            countries_list.append(split_line[0:3])
    countries =  pd.DataFrame(countries_list, columns=["id", "code" ,"name"])
    countries["id"] = pd.to_numeric(countries["id"])
    countries = countries.set_index("id")
    return countries
countries = extract_gnwo_countries_to_df()

display(countries.head(1))

,code,name
id,,
2,USA,United States of America


In [529]:
def get_country_name(gwno):
    try:
        name = countries.loc[gwno, "name"]
    except:
        name = "N/A"
    return name

In [530]:
def get_popup_icon_style_from_conflict_type(pd_series, size=5):
    """Really basic function to get the icon style from the type of series, could be more complex""" 
    BASE_MARKER_NAME="free-map-marker-icon-"
    icon_style_dict_array = []
    for index, value in enumerate(pd_series):
        # Add more values as needed
        icon_style_base_dict = {}
        icon_style_base_dict['iconSize'] = [size, size]
        local_dict = icon_style_base_dict
        if value == 1:
            color = 'blue'
        elif value == 2:
            color = 'orange'
        elif value == 3:
            color = 'pink'
        #local_dict['iconColor'] = color
        icon_name = BASE_MARKER_NAME+color+".png"
        local_dict['iconUrl'] = os.path.join("../data", "icons", icon_name)#"http://besticons.net/sites/default/files/chat-exclamation-icon-8614.png"
        local_dict['iconOpacity'] = 0.1
        local_dict['iconAnchor'] = (size//2,size)
        icon_style_dict_array.append(local_dict)
    return icon_style_dict_array

In [531]:
base_popup_table_str = """
        <h3>Conflict Id: {}<h3>
        <h4>Conflict Type: {}<h4>
        <table style=\"width:100%\">
          <tr>
            <th>Side</th>
            <th>Actor Name</th> 
            <th>Country</th>
          </tr>
          <tr>
            <td>Side A</td>
            <td>{}</td> 
            <td>{}</td>
          </tr>
          <tr>
            <td>Side B</td>
            <td>{}</td> 
            <td>{}</td>
          </tr>
        </table>
        <table style=\"width:100%\">
          <tr>
            <th>Actor</th>
            <th>Deaths</th> 
          </tr>
          <tr>
            <td>Side A</td>
            <td>{}</td> 
          </tr>
          <tr>
            <td>Side B</td>
            <td>{}</td> 
          </tr>
          <tr>
            <td>Civilians</td>
            <td>{}</td> 
          </tr>
          <tr>
            <td>Total</td>
            <td>{}</td> 
          </tr
        </table>
    """

In [532]:
conflict_type_from_nbr = {
    1:"State Based",
    2:"Non State",
    3:"One Sided",
}

In [533]:
def get_feature_marker_array(df):
    """This function uses the extracted features to create the feature dictionnary"""
    marker_style_array = get_popup_icon_style_from_conflict_type(df.type_of_violence, size=20)
    features = []
    for row in tqdm(df.itertuples()):
        if row.best is 0:
            continue
        local_marker_feature_dict = {
           "type": "Feature",
           "geometry": {"type": "Point",
                        "coordinates": [row.longitude, row.latitude]
                       },
           "properties" : {"time": "{}-01-01".format(int(row.year)),
                           #"popup": "<h1>{}</h1>".format(np.asscalar(row.conflict_id)),
                           "popup": base_popup_table_str.format(np.asscalar(row.conflict_id),
                                          conflict_type_from_nbr[row.type_of_violence],
                                          row.side_a,
                                          get_country_name(row.gwnoa),
                                          row.side_b,
                                          get_country_name(row.gwnob),
                                          np.asscalar(row.deaths_a),
                                          np.asscalar(row.deaths_b),
                                          np.asscalar(row.deaths_civilians),
                                          np.asscalar(row.best)
                                    ),
                           "icon": "marker",
                           "iconstyle": marker_style_array[row.Index],
                           "id":"marker_{}".format(np.asscalar(row.Index))

           }
        }
        features.append(local_marker_feature_dict)
    return features
        

In [534]:
feature_marker_array = get_feature_marker_array(conflict_graph_df)

4865it [00:00, 5873.78it/s]


In [307]:
print(feature_marker_array[0])

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [45.931985114754106, 14.257709180327867]}, 'properties': {'time': '2010-01-01', 'popup': '<h1>230.0</h1>', 'icon': 'marker', 'iconstyle': {'iconSize': [5, 5], 'iconColor': 'cadetblue', 'icon': <folium.map.Icon object at 0x7f2a3cefc630>}, 'id': 'marker_0'}}


#### Create feature for the various countries metrics

#### Create Map

In [535]:
m = folium.Map(tiles='cartodbpositron', world_copy_jump = True, no_wrap=True, zoom_start=5)
feature_group = folium.FeatureGroup(name='Total Deaths')

total_deaths_circle = plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': feature_circle_array[0]
    },
    period='P1Y',
    add_last_point=False,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY',
    time_slider_drag_update=True,
)
m.add_child(total_deaths_circle, name="Total Deaths")

civilian_deaths_circle = plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': feature_circle_array[1]
    },
    period='P1Y',
    add_last_point=False,
    auto_play=False,
    loop=False,
    max_speed=10,
    loop_button=True,
    date_options='YYYY',
    time_slider_drag_update=True,
)
m.add_child(civilian_deaths_circle, name="Civilian Deaths")

popup_markers = plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': feature_marker_array 
    },
    period='P1Y',
    add_last_point=False,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY',
    time_slider_drag_update=True,
)
m.add_child(popup_markers, name="Conflict Markers")


folium.LayerControl().add_to(m)

m.save(os.path.join('results', "map_conflict_feature_geojson.html"))

In [536]:
# HACK: We do this because of the limitations of the Folium library
# NOTE: need to add: duration: period in the html script of the TimestampedGeoJson plugin
#                       at the line 147, in geoJson Layer instanciation
# First hack to have good refresh
replace_period = '"P11M"'
with fileinput.FileInput(os.path.join('results', "map_conflict_feature_geojson.html"), inplace=True, backup='.back') as f:
    TEXT_TO_SEARCH = "updateTimeDimension: "
    TEXT_TO_REPLACE = "duration: {}, {}".format(replace_period, TEXT_TO_SEARCH)
    for line in f:
        print(line.replace(TEXT_TO_SEARCH, TEXT_TO_REPLACE), end='')
 
# Second hack to remove multiple occurences of the TimeDimension Control
with fileinput.FileInput(os.path.join('results', "map_conflict_feature_geojson.html"), inplace=True, backup='.back') as f:      
    counter = 0
    TEXT_TO_SEARCH = "L.Control.TimeDimensionCustom = L.Control.TimeDimension.extend"
    for line_nbr, line in enumerate(f):
        if line_nbr < 128: # End of first map layer
            print(line, end='')
        # Skip lines starting from the TimeDimensionCustom
        elif TEXT_TO_SEARCH in line:
            print("//{}".format(line),end='')
            counter += 1
            continue
        elif 0 < counter < 22:
            print("//{}".format(line),end='')
            counter += 1
        else:
            counter = 0
            print(line,end='')
